In [1]:
import simpy
from numpy.random import choice
#from random import expovariate
import numpy as np

In [2]:
n=5 #Prime router number
N=10 #regular router number
m=3 #shortest path number

In [3]:
file_name_shortest_path='full_graph_shortest_path_{}_{}.txt'.format(n,N)
file_name_prime_graph='prime_graph_{}_{}.txt'.format(n,N)
file_name_prime_graph_optimal_path='prime_graph_optimal_path_{}_{}.txt'.format(n,N)
file_best_clustering='full_graph_best_clustering_{}_{}.txt'.format(n,N)
file_name_cluster_graph='cluster_graph_{}_{}.txt'.format(n,N)

# Reading clustering and prime graphs

In [4]:
con=[[] for _ in range(n+N+1)]
edges_clust=set()

In [5]:
with open('test/'+file_name_cluster_graph, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    #f.write('\n')
    for line in f:
        #print('line',line)
        x,y,w=map(float,line.split())
        x,y=int(x),int(y)
        #print(x,y,w)
        con[x].append((y,w))
        #con[y].append((x,w))
        edges_clust.add((x,y,w))

           
    f.close()

In [6]:
edges_clust

{(1, 3, 0.23142470169810703),
 (1, 5, 0.7536451376844908),
 (1, 6, 0.4638092727662523),
 (1, 7, 0.9581887310477116),
 (1, 8, 0.11962100470551662),
 (1, 13, 0.10916629732677441),
 (1, 14, 0.9991049222877524),
 (2, 3, 0.032934312265980425),
 (2, 5, 0.6369470115993228),
 (2, 6, 0.9557802366404332),
 (2, 9, 0.3430049053278973),
 (2, 14, 0.7731763192800118),
 (3, 1, 0.23142470169810703),
 (3, 2, 0.032934312265980425),
 (3, 4, 0.501869175806194),
 (3, 5, 0.882580654303218),
 (3, 6, 0.013962574310672937),
 (3, 7, 0.3114343887508788),
 (3, 11, 0.6107640985517377),
 (3, 12, 0.7868252034308975),
 (3, 14, 0.9658427179915304),
 (4, 3, 0.501869175806194),
 (4, 6, 0.7856803587257708),
 (4, 8, 0.6293629338431845),
 (4, 9, 0.15686755357253557),
 (4, 13, 0.959083650317909),
 (5, 1, 0.7536451376844908),
 (5, 2, 0.6369470115993228),
 (5, 3, 0.882580654303218),
 (5, 11, 0.527469403352322),
 (5, 12, 0.9106297271042973),
 (5, 13, 0.2205953804350782),
 (6, 1, 0.4638092727662523),
 (6, 2, 0.9557802366404332),

In [7]:
con=[[] for _ in range(n+N+1)]
edges_prime=set()

In [8]:
with open('test/'+file_name_prime_graph, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    #f.write('\n')
    for line in f:
        #print('line',line)
        x,y,w=map(float,line.split())
        x,y=int(x),int(y)
        #print(x,y,w)
        con[x+N].append((y+N,w))
        #con[y].append((x,w))
        edges_prime.add((x+N,y+N,w))

           
    f.close()

In [9]:
edges_prime

{(11, 12, 6.089308538078741),
 (12, 11, 6.089308538078741),
 (12, 13, 3.0),
 (12, 14, 4.977825363992857),
 (12, 15, 5.3342404165033095),
 (13, 12, 3.0),
 (13, 15, 3.1448932613183276),
 (14, 12, 4.977825363992857),
 (15, 12, 5.3342404165033095),
 (15, 13, 3.1448932613183276)}

In [10]:
edges=edges_clust.union(edges_prime)

In [11]:
edges

{(1, 3, 0.23142470169810703),
 (1, 5, 0.7536451376844908),
 (1, 6, 0.4638092727662523),
 (1, 7, 0.9581887310477116),
 (1, 8, 0.11962100470551662),
 (1, 13, 0.10916629732677441),
 (1, 14, 0.9991049222877524),
 (2, 3, 0.032934312265980425),
 (2, 5, 0.6369470115993228),
 (2, 6, 0.9557802366404332),
 (2, 9, 0.3430049053278973),
 (2, 14, 0.7731763192800118),
 (3, 1, 0.23142470169810703),
 (3, 2, 0.032934312265980425),
 (3, 4, 0.501869175806194),
 (3, 5, 0.882580654303218),
 (3, 6, 0.013962574310672937),
 (3, 7, 0.3114343887508788),
 (3, 11, 0.6107640985517377),
 (3, 12, 0.7868252034308975),
 (3, 14, 0.9658427179915304),
 (4, 3, 0.501869175806194),
 (4, 6, 0.7856803587257708),
 (4, 8, 0.6293629338431845),
 (4, 9, 0.15686755357253557),
 (4, 13, 0.959083650317909),
 (5, 1, 0.7536451376844908),
 (5, 2, 0.6369470115993228),
 (5, 3, 0.882580654303218),
 (5, 11, 0.527469403352322),
 (5, 12, 0.9106297271042973),
 (5, 13, 0.2205953804350782),
 (6, 1, 0.4638092727662523),
 (6, 2, 0.9557802366404332),

# Reading optimal clustering

In [12]:
clust=[0]*(N+1)

reverse_clust=[set() for i in range(n+N+1)] 

clust_opt_path=[0]*(N+1)

In [13]:
with open('test/'+file_best_clustering, 'r') as f:
    
    n,N=map(int,f.readline().split())
    next(f)
    
    i=1
    line_cnt=0
    for line in f:
        line_cnt+=1 
        
        if line_cnt%3:
            if line_cnt%3==1:
                i,j=map(int,line.split())
                clust[i]=j
                reverse_clust[j].add(i)
            else:
                
                path = list(map(int,line.split()))
                
                #print('path',path)
                clust_opt_path[i]=path
                i+=1
    f.close()

In [14]:
n

5

In [15]:
clust[1:]

[15, 14, 14, 12, 12, 12, 14, 13, 11, 14]

In [16]:
reverse_clust[N+1:]

[{9}, {4, 5, 6}, {8}, {2, 3, 7, 10}, {1}]

In [17]:
clust_opt_path[1:]

[[1, 15],
 [2, 9, 7, 14],
 [3, 14],
 [4, 12],
 [5, 2, 12],
 [6, 12],
 [7, 14],
 [8, 7, 9, 13],
 [9, 11],
 [10, 8, 7, 14]]

# Reading optimal routing from prime graph

In [18]:
prime_opt_path=[0]*(n+1)

for i in range(n+1):
    prime_opt_path[i]=[0]*(n+1)

In [19]:
with open('test/'+file_name_prime_graph_optimal_path, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    line_cnt=0
    for line in f:
        line_cnt+=1 
        
        if line_cnt%3:
            
            if line_cnt%3==1:
                #print('line',line)
                i,j=map(int,line.split())
            else:
                path = list(map(int,line.split()))
                #print('path',path)

                prime_opt_path[i][j]=np.array(path)+N
        
    f.close()

In [20]:
prime_opt_path

[[0, 0, 0, 0, 0, 0],
 [0,
  0,
  array([11, 12]),
  array([11, 12, 13]),
  array([11, 12, 14]),
  array([11, 12, 13, 15])],
 [0,
  array([12, 11]),
  0,
  array([12, 13]),
  array([12, 14]),
  array([12, 13, 15])],
 [0,
  array([13, 15, 12, 11]),
  array([13, 12]),
  0,
  array([13, 15, 12, 14]),
  array([13, 12, 15])],
 [0,
  array([14, 12, 11]),
  array([14, 12]),
  array([14, 12, 15, 13]),
  0,
  array([14, 12, 13, 15])],
 [0,
  array([15, 12, 11]),
  array([15, 13, 12]),
  array([15, 13]),
  array([15, 13, 12, 14]),
  0]]

In [21]:
clust[1]

15

In [22]:
clust[2]

14

In [23]:
clust_opt_path

[0,
 [1, 15],
 [2, 9, 7, 14],
 [3, 14],
 [4, 12],
 [5, 2, 12],
 [6, 12],
 [7, 14],
 [8, 7, 9, 13],
 [9, 11],
 [10, 8, 7, 14]]

# Optimal routing in full graph

In [24]:
full_graph_opt_path=[0]*(N+1)

for i in range(1,N+1):
    full_graph_opt_path[i]=[0]*(N+1)

In [25]:
clust_opt_path[1]

[1, 15]

In [26]:
clust_opt_path[2]

[2, 9, 7, 14]

In [27]:
prime_opt_path[4][5]

array([14, 12, 13, 15])

In [28]:
prime_opt_path[2][3]

array([12, 13])

In [29]:
for i in range(1,N+1):
    for j in range(1,N+1):
        if i != j:
            prime_i=clust[i]
            prime_j=clust[j]
            
            if prime_i==prime_j:
                full_graph_opt_path[i][j]=[]
                continue
            
            #print('hhh',i,j,prime_i,prime_j)
            
            prime_path=prime_opt_path[prime_i-N][prime_j-N]
            #print(i,j, 'prime_path!',prime_path)
            
            l=len(prime_path)
            #print(i,j,clust_opt_path[i],prime_path[1:l-1],clust_opt_path[j][::-1])
            
            
            full_graph_opt_path[i][j]=clust_opt_path[i]+list(prime_path[1:l-1])+clust_opt_path[j][::-1]
            
            

# Network simulation

In [30]:
class Packet(object):
    """ A very simple class that represents a packet.
        This packet will run through a queue at a switch output port.
        We use a float to represent the size of the packet in bytes so that
        we can compare to ideal M/M/1 queues.

        Parameters
        ----------
        
        packet_id : int
            an identifier for the packet
            
        path : list
            path from source to destination
            
        cur_path_ind : int
            current path position
        
        time : float
            the time the packet arrives at the output queue
            
        size : float
            the size of the packet in bytes
        
        src, dst : int
            identifiers for the source and the destination

    """
    def __init__(self, packet_id, path, cur_path_ind, time, size, src, dst):
        
        self.packet_id=packet_id
        self.path=path
        self.cur_path_ind=cur_path_ind
        
        
        self.time = time
        self.size = size
        self.src = src
        self.dst = dst


    def __repr__(self):
        return "id: {}, src: {}, time: {}, size: {}".\
            format(self.packet_id, self.src, self.time, self.size)

In [31]:
class PacketGenerator(object):
    """ Generates packets with given inter-arrival time distribution. 
        Set the "out" member variable to the entity to receive the packet.
        Then it generates a destination (for each package independently)
        according to predetermined distribution and sends packages along the predetemined paths.
        PacketGenerator i is connected to Switch i.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
            
        gen_prob : list
            destination probability distribution
            
        generator_id : int
            PacketGenerator id
            
        adist : function
            a no parameter function that returns the successive inter-arrival times of the packets
            
        sdist : function
            a no parameter function that returns the successive sizes of the packets
            
        initial_delay : int
            Starts generation after an initial delay. Default = 0
            
        finish : int
            Stops generation at the finish time. Default is infinite


    """
    def __init__(self, env, gen_prob, generator_id, adist, sdist, initial_delay=0, finish=float("inf")):
        
        self.gen_prob=gen_prob
        self.generator_id = generator_id
        self.env = env
        self.adist = adist
        self.sdist = sdist
        self.initial_delay = initial_delay
        self.finish = finish
        self.packets_sent = 0
        self.action = env.process(self.run())  # starts the run() method as a SimPy process


    def run(self):
        """The generator function used in simulations.
        """
        
        yield self.env.timeout(self.initial_delay)
        while self.env.now < self.finish:
            yield self.env.timeout(self.adist())
            
            destination = choice(range(1,N+1), p=self.gen_prob)
            opt_path=full_graph_opt_path[self.generator_id][destination]

            self.packets_sent += 1
            
            pkt = Packet(packet_id=self.packets_sent, 
                         path=opt_path, 
                         cur_path_ind=0, 
                         time=self.env.now, 
                         size=self.sdist(), 
                         src=self.generator_id, 
                         dst=destination)

            
            self.out.put(pkt)
       

In [32]:
class PacketSink(object):
    """ Receives packets and collects delay information into the
        waits list. You can then use this list to look at delay statistics.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
            
        debug : boolean
            if true then the contents of each packet will be printed as it is received.
            
        rec_arrivals : boolean
            if true then arrivals will be recorded
            
        absolute_arrivals : boolean
            if true absolute arrival times will be recorded, otherwise the time between consecutive arrivals
            is recorded.
            
        rec_waits : boolean
            if true waiting time experienced by each packet is recorded
            

    """
    def __init__(self, env, rec_arrivals=False, absolute_arrivals=False, rec_waits=True, debug=False):
        self.store = simpy.Store(env)
        self.env = env
        self.rec_waits = rec_waits
        self.rec_arrivals = rec_arrivals
        self.absolute_arrivals = absolute_arrivals
        self.waits = []
        self.arrivals = []
        self.debug = debug
        self.packets_rec = 0
        self.bytes_rec = 0
        self.last_arrival = 0.0

    def put(self, pkt):

        now = self.env.now
        
        if self.rec_waits:
            self.waits.append(self.env.now - pkt.time)
        if self.rec_arrivals:
            if self.absolute_arrivals:
                self.arrivals.append(now)
            else:
                self.arrivals.append(now - self.last_arrival)
            self.last_arrival = now
        self.packets_rec += 1
        self.bytes_rec += pkt.size


In [33]:
class SwitchPort(object):
    """ Models a switch output port with a given rate and buffer size limit in bytes.
        First, it stores packets in queue (put method), then it extracts them one-by-one 
        and sends to the next switch or sink.
        

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        rate : float
            the bit rate of the port
        qlimit : integer (or None)
            a buffer size limit in bytes or packets for the queue (including items
            in service).
        limit_bytes : If true, the queue limit will be based on bytes if false the
            queue limit will be based on packets.

    """
    def __init__(self, env, router_id, rate, qlimit=None, limit_bytes=True, debug=False):
        self.router_id=router_id
        self.store = simpy.Store(env)
        self.rate = rate
        self.env = env
        self.out = None
        self.packets_rec = 0
        self.packets_drop = 0
        self.qlimit = qlimit
        self.limit_bytes = limit_bytes
        self.byte_size = 0  # Current size of the queue in bytes
        self.debug = debug
        self.busy = 0  # Used to track if a packet is currently being sent
        self.action = env.process(self.run())  # starts the run() method as a SimPy process

    def run(self):

        while True:
            msg = (yield self.store.get())
            self.busy = 1
            self.byte_size -= msg.size
            yield self.env.timeout(msg.size*8.0/self.rate)
            
            path=msg.path
            cur_path_ind=msg.cur_path_ind
            destination=msg.dst
                
                
            if path[cur_path_ind]==destination:
                
                sink.put(msg)
            else:
                next_node=path[cur_path_ind+1]
                msg.cur_path_ind=cur_path_ind+1
                
                #print('',)
                #print('next_node',next_node)
                #print('switch[next_node]',switch[next_node])
                switch[next_node].store.put(msg)   
                
                
                
            self.busy = 0
            if self.debug:
                print(msg)
                
     

    def put(self, pkt):

        self.packets_rec += 1
        tmp_byte_count = self.byte_size + pkt.size

        if self.limit_bytes and tmp_byte_count >= self.qlimit:
            self.packets_drop += 1
            return
        
        else:
            self.byte_size = tmp_byte_count
            
            return self.store.put(pkt)    
                

In [34]:
def RandomizedArrival():
    #return 1.5    # time interval
    return max(np.random.normal(1.5, 0.5),1)

def constSize():
    return 100.0  # bytes

In [35]:

prob=[0]*4
for i in range(4):
    path[i]=[0]*4



gen_prob=[0]*4
gen_prob[1]=[0,0.5,0.5]
gen_prob[2]=[0.2,0,0.8]
gen_prob[3]=[0.7,0.3,0]

In [36]:
gen_prob=[0]*(N+1)

for i in range(1,N+1):
    prime_router=clust[i]
    #print('reverse_clust(prime_router)',reverse_clust[prime_router])
    clust_len=len(reverse_clust[prime_router])
    
    gen_prob[i]=[1./(N-clust_len) if not j+1 in reverse_clust[prime_router] else 0 for j in range(N)]
    


In [37]:
gen_prob

[0,
 [0,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111,
  0.1111111111111111],
 [0.16666666666666666,
  0,
  0,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0,
  0.16666666666666666,
  0.16666666666666666,
  0],
 [0.16666666666666666,
  0,
  0,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666,
  0,
  0.16666666666666666,
  0.16666666666666666,
  0],
 [0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0,
  0,
  0,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285],
 [0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0,
  0,
  0,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285],
 [0.14285714285714285,
  0.14285714285714285,
  0.14285714285714285,
  0,
  0,
  0,
  0.14285714285714285,
  0.142857142

In [38]:
generators=[0]*(N+1)
switch=[0]*(n+N+1)

env = simpy.Environment()  # Create the SimPy environment

sink=PacketSink(env, debug=True)

for i in range(1,n+N+1):
    switch[i]=SwitchPort(env=env, router_id=i, rate=1000.0, qlimit=3000, debug=True)

for i in range(1,N+1):
    generators[i]=PacketGenerator(env, gen_prob[i], i, RandomizedArrival, constSize)
    generators[i].out = switch[i]


    
env.run(until=2000)
print("average delivery time:", format(sum(sink.waits)/max(1,sink.packets_rec)))
print('number of delivered packages:', sink.packets_rec)
print('number of packages stuck in the network:\n')

for i in range(1,n+N+1):
    print('router {} packages {}'.format(i,len(switch[i].store.items)))

print('\n')
for i in range(1,n+N+1):
    print("router {} has dropped {} packages ".format(i,switch[i].packets_drop))

print('\n')
for i in range(1,N+1):        
    print('generator {} has sent {} packages'.format(i, generators[i].packets_sent))


id: 1, src: 5, time: 1, size: 100.0
id: 1, src: 8, time: 1, size: 100.0
id: 1, src: 7, time: 1.24447748315456, size: 100.0
id: 1, src: 6, time: 1.2524611943084116, size: 100.0
id: 1, src: 10, time: 1.334684676570122, size: 100.0
id: 1, src: 9, time: 1.6175452500743666, size: 100.0
id: 1, src: 2, time: 1.7128463971965686, size: 100.0
id: 1, src: 1, time: 1.7640866657176608, size: 100.0
id: 1, src: 4, time: 1.9155371254326115, size: 100.0
id: 1, src: 8, time: 1, size: 100.0
id: 1, src: 7, time: 1.24447748315456, size: 100.0
id: 1, src: 6, time: 1.2524611943084116, size: 100.0
id: 1, src: 3, time: 2.1012172540074148, size: 100.0
id: 1, src: 10, time: 1.334684676570122, size: 100.0
id: 1, src: 9, time: 1.6175452500743666, size: 100.0
id: 2, src: 6, time: 2.4654944920923594, size: 100.0
id: 1, src: 5, time: 1, size: 100.0
id: 1, src: 2, time: 1.7128463971965686, size: 100.0
id: 1, src: 1, time: 1.7640866657176608, size: 100.0
id: 2, src: 5, time: 2.6699282975498937, size: 100.0
id: 2, src: 

id: 11, src: 4, time: 16.324109127383135, size: 100.0
id: 19, src: 9, time: 27.049331891021787, size: 100.0
id: 49, src: 3, time: 81.36012038873208, size: 100.0
id: 80, src: 10, time: 122.9595476348187, size: 100.0
id: 80, src: 1, time: 120.77250897689325, size: 100.0
id: 81, src: 1, time: 122.32715041306876, size: 100.0
id: 81, src: 5, time: 123.15609663383808, size: 100.0
id: 17, src: 6, time: 27.843996677094456, size: 100.0
id: 66, src: 10, time: 102.2087622608932, size: 100.0
id: 8, src: 7, time: 13.58700431834296, size: 100.0
id: 43, src: 2, time: 69.89578998194297, size: 100.0
id: 9, src: 7, time: 15.434854144879557, size: 100.0
id: 38, src: 8, time: 55.875123410920935, size: 100.0
id: 28, src: 1, time: 39.975860607149095, size: 100.0
id: 13, src: 9, time: 18.970852133844794, size: 100.0
id: 78, src: 6, time: 123.83107435873404, size: 100.0
id: 81, src: 10, time: 123.9595476348187, size: 100.0
id: 75, src: 3, time: 123.87813964592094, size: 100.0
id: 80, src: 4, time: 124.0158990

id: 135, src: 5, time: 207.16726853882184, size: 100.0
id: 150, src: 10, time: 234.52013709488443, size: 100.0
id: 33, src: 6, time: 52.762097191556684, size: 100.0
id: 78, src: 7, time: 126.7838460699649, size: 100.0
id: 45, src: 5, time: 67.4220742001496, size: 100.0
id: 54, src: 7, time: 88.60810695446692, size: 100.0
id: 49, src: 8, time: 72.45542707940633, size: 100.0
id: 150, src: 1, time: 233.4402610133899, size: 100.0
id: 138, src: 8, time: 199.9672787930322, size: 100.0
id: 151, src: 1, time: 235.26829057879954, size: 100.0
id: 153, src: 5, time: 235.29855435712673, size: 100.0
id: 146, src: 6, time: 235.3001962306396, size: 100.0
id: 129, src: 2, time: 208.61240631548802, size: 100.0
id: 82, src: 2, time: 136.69236492695114, size: 100.0
id: 153, src: 4, time: 235.4681334973638, size: 100.0
id: 151, src: 10, time: 235.5379021264183, size: 100.0
id: 19, src: 4, time: 29.243292293621806, size: 100.0
id: 52, src: 1, time: 78.65800844873905, size: 100.0
id: 100, src: 3, time: 161.

id: 222, src: 4, time: 346.7810668763143, size: 100.0
id: 52, src: 9, time: 77.31411613239787, size: 100.0
id: 115, src: 7, time: 186.81480603039466, size: 100.0
id: 24, src: 7, time: 39.329425273579375, size: 100.0
id: 46, src: 5, time: 68.90783949289545, size: 100.0
id: 74, src: 4, time: 113.67255538530947, size: 100.0
id: 222, src: 5, time: 346.87544853788717, size: 100.0
id: 219, src: 3, time: 347.0026189794773, size: 100.0
id: 203, src: 8, time: 300.64309016547065, size: 100.0
id: 195, src: 2, time: 305.0483715916441, size: 100.0
id: 148, src: 1, time: 230.03457472739314, size: 100.0
id: 221, src: 6, time: 347.57334302784693, size: 100.0
id: 25, src: 6, time: 42.142249520182034, size: 100.0
id: 74, src: 1, time: 112.0951208991722, size: 100.0
id: 152, src: 3, time: 242.81783795795258, size: 100.0
id: 226, src: 1, time: 345.9881612710652, size: 100.0
id: 74, src: 4, time: 113.67255538530947, size: 100.0
id: 223, src: 4, time: 347.7810668763143, size: 100.0
id: 223, src: 5, time: 34

id: 298, src: 3, time: 468.01699303880025, size: 100.0
id: 261, src: 10, time: 404.5858488645849, size: 100.0
id: 301, src: 10, time: 468.0515685969858, size: 100.0
id: 302, src: 6, time: 468.09961073037516, size: 100.0
id: 267, src: 5, time: 412.46392129210204, size: 100.0
id: 173, src: 2, time: 271.26256831487285, size: 100.0
id: 88, src: 5, time: 133.48288986089264, size: 100.0
id: 308, src: 1, time: 467.6039074765888, size: 100.0
id: 147, src: 8, time: 212.97860394594335, size: 100.0
id: 29, src: 8, time: 45.021872627785825, size: 100.0
id: 108, src: 7, time: 173.95026270404577, size: 100.0
id: 34, src: 7, time: 56.69835595037164, size: 100.0
id: 303, src: 5, time: 468.60836383305156, size: 100.0
id: 62, src: 5, time: 91.89599817704763, size: 100.0
id: 274, src: 8, time: 405.8599380390624, size: 100.0
id: 265, src: 2, time: 413.5882917877786, size: 100.0
id: 207, src: 9, time: 314.3483390055372, size: 100.0
id: 307, src: 1, time: 465.992845842747, size: 100.0
id: 88, src: 5, time: 

...

id: 311, src: 8, time: 461.76424142733913, size: 100.0
id: 651, src: 9, time: 1001.927776635242, size: 100.0
id: 984, src: 6, time: 1513.0130767914918, size: 100.0
id: 464, src: 8, time: 704.1766627035702, size: 100.0
id: 315, src: 6, time: 488.2578285511543, size: 100.0
id: 211, src: 1, time: 324.52041529429135, size: 100.0
id: 984, src: 1, time: 1512.6155422074978, size: 100.0
id: 982, src: 10, time: 1513.5354994591123, size: 100.0
id: 980, src: 5, time: 1513.599251083465, size: 100.0
id: 983, src: 1, time: 1510.6051339224475, size: 100.0
id: 876, src: 8, time: 1338.684233877426, size: 100.0
id: 865, src: 5, time: 1338.8553238043182, size: 100.0
id: 985, src: 1, time: 1513.7814433490853, size: 100.0
id: 965, src: 4, time: 1513.782545614933, size: 100.0
id: 652, src: 9, time: 1002.927776635242, size: 100.0
id: 215, src: 9, time: 325.7119940087501, size: 100.0
id: 508, src: 7, time: 794.9425300775847, size: 100.0
id: 99, src: 10, time: 153.21561469573982, size: 100.0
id: 649, src: 3, t

id: 368, src: 7, time: 574.256292329807, size: 100.0
id: 699, src: 9, time: 1077.7555940775683, size: 100.0
id: 1042, src: 4, time: 1634.0594759623211, size: 100.0
id: 1060, src: 1, time: 1630.168037018507, size: 100.0
id: 300, src: 5, time: 462.79212591481996, size: 100.0
id: 1062, src: 1, time: 1632.168037018507, size: 100.0
id: 1062, src: 10, time: 1634.0649112743406, size: 100.0
id: 940, src: 10, time: 1444.9386677705659, size: 100.0
id: 935, src: 5, time: 1445.0385067549173, size: 100.0
id: 700, src: 9, time: 1078.7555940775683, size: 100.0
id: 322, src: 2, time: 504.66870674208167, size: 100.0
id: 301, src: 5, time: 463.79212591481996, size: 100.0
id: 321, src: 10, time: 498.61032130757565, size: 100.0
id: 67, src: 2, time: 110.91696771817601, size: 100.0
id: 339, src: 6, time: 527.1121017393153, size: 100.0
id: 1063, src: 1, time: 1633.446720509825, size: 100.0
id: 1066, src: 3, time: 1635.2100275649432, size: 100.0
id: 1058, src: 5, time: 1635.222004970804, size: 100.0
id: 941,

id: 749, src: 9, time: 1152.5541380040152, size: 100.0
id: 1132, src: 6, time: 1745.8553630674342, size: 100.0
id: 1136, src: 1, time: 1744.431828274649, size: 100.0
id: 124, src: 6, time: 200.12874385734972, size: 100.0
id: 128, src: 7, time: 208.2342128648482, size: 100.0
id: 749, src: 3, time: 1159.9722128918168, size: 100.0
id: 1142, src: 3, time: 1746.359666053857, size: 100.0
id: 1002, src: 10, time: 1543.2233415830483, size: 100.0
id: 989, src: 2, time: 1546.04251415528, size: 100.0
id: 1137, src: 1, time: 1745.7832795417999, size: 100.0
id: 756, src: 1, time: 1150.5355586306798, size: 100.0
id: 749, src: 9, time: 1152.5541380040152, size: 100.0
id: 1115, src: 4, time: 1746.72032689002, size: 100.0
id: 1132, src: 10, time: 1746.7679777763772, size: 100.0
id: 1138, src: 1, time: 1746.7832795417999, size: 100.0
id: 528, src: 10, time: 810.9580239275193, size: 100.0
id: 365, src: 6, time: 566.6921607994233, size: 100.0
id: 393, src: 7, time: 611.7867763890824, size: 100.0
id: 128, 

id: 1207, src: 10, time: 1860.7361738773732, size: 100.0
id: 1068, src: 10, time: 1644.7286846408028, size: 100.0
id: 1064, src: 5, time: 1645.000781593951, size: 100.0
id: 107, src: 10, time: 165.25426409183217, size: 100.0
id: 378, src: 8, time: 567.9463540392182, size: 100.0
id: 799, src: 9, time: 1230.4688288917196, size: 100.0
id: 262, src: 3, time: 411.14345034716405, size: 100.0
id: 1221, src: 3, time: 1861.1083091277333, size: 100.0
id: 1189, src: 4, time: 1861.1140403329534, size: 100.0
id: 1205, src: 6, time: 1861.1500830143402, size: 100.0
id: 97, src: 9, time: 144.77681818997002, size: 100.0
id: 262, src: 9, time: 402.44936020710895, size: 100.0
id: 418, src: 7, time: 648.4612213626598, size: 100.0
id: 1212, src: 1, time: 1861.4556687335175, size: 100.0
id: 1075, src: 8, time: 1646.6539100538132, size: 100.0
id: 1054, src: 2, time: 1646.1139514429824, size: 100.0
id: 1205, src: 5, time: 1861.3570694185976, size: 100.0
id: 179, src: 9, time: 267.4995324288773, size: 100.0
id

id: 1127, src: 10, time: 1739.5813134413684, size: 100.0
id: 1116, src: 2, time: 1739.171102817843, size: 100.0
id: 848, src: 9, time: 1304.922265427005, size: 100.0
id: 126, src: 10, time: 197.26864063914414, size: 100.0
id: 1277, src: 1, time: 1963.3997667382243, size: 100.0
id: 1276, src: 1, time: 1961.7549895837624, size: 100.0
id: 601, src: 8, time: 912.4648602033167, size: 100.0
id: 144, src: 7, time: 233.40411707843126, size: 100.0
id: 844, src: 3, time: 1299.8684587434877, size: 100.0
id: 1270, src: 6, time: 1964.6884374306665, size: 100.0
id: 1272, src: 10, time: 1964.7732505487213, size: 100.0
id: 1290, src: 3, time: 1964.8178739389123, size: 100.0
id: 1135, src: 8, time: 1741.7081403300633, size: 100.0
id: 1126, src: 5, time: 1739.1933960052982, size: 100.0
id: 96, src: 7, time: 155.58696644687473, size: 100.0
id: 848, src: 9, time: 1304.922265427005, size: 100.0
id: 277, src: 9, time: 422.4744338563217, size: 100.0
id: 1275, src: 5, time: 1965.0577643085226, size: 100.0
id: